# A Guided Tour of Ray Core: Remote Objects

[*Remote Objects*](https://docs.ray.io/en/latest/walkthrough.html#objects-in-ray)
implement a [*shared-memory object store*](https://en.wikipedia.org/wiki/Shared_memory) pattern.

Objects are immutable, and can be accessed from anywhere on the cluster.

---

First, let's start Ray…

In [1]:
from icecream import ic
import logging
import ray

ray.init(
    ignore_reinit_error=True,
    logging_level=logging.ERROR,
)

print(f"Dashboard URL: http://{ray.get_dashboard_url()}")

Dashboard URL: http://127.0.0.1:8265


## Remote Objects example

Set up for this example...

In [2]:
import time

To start, we'll put an object into the Ray object store...

In [3]:
y = 42
obj_ref = ray.put(y)

Then get the value of this object reference

In [4]:
ray.get(obj_ref)

42

You can also access the values of multiple object references in parallel:

In [5]:
ray.get([ray.put(i) for i in range(3)])

[0, 1, 2]

Now let's set a timeout to return early from a `get` that is blocking for too long...

In [6]:
from ray.exceptions import GetTimeoutError

@ray.remote
def long_running_function ():
    time.sleep(4)
    return 42

In [7]:
obj_ref = long_running_function.remote()

try:
    ray.get(obj_ref, timeout=4)
except GetTimeoutError:
    print("`get` timed out")

`get` timed out


Then shutdown Ray

In [8]:
ray.shutdown()